In [5]:
import cx_Oracle
import psycopg2
from psycopg2 import sql
import os

In [10]:
# === Oracle connection ===
ORACLE = {
    "user": "hr",
    "password": "hr",
    "dsn": "localhost:1521/orclpdb1",
}

# === PostgreSQL connection ===
POSTGRES = {
    "dbname": "mypgdb",
    "user": "admin",
    "password": "admin123",
    "host": "localhost",
    "port": 5432,
}

CHUNK_SIZE = 1000

In [15]:

def map_oracle_type(ora_type):
    # Simplified type mapper
    if 'CHAR' in ora_type or 'CLOB' in ora_type:
        return 'TEXT'
    elif 'NUMBER' in ora_type or 'FLOAT' in ora_type:
        return 'NUMERIC'
    elif 'DATE' in ora_type or 'TIMESTAMP' in ora_type:
        return 'TIMESTAMP'
    elif 'BLOB' in ora_type:
        return 'BYTEA'
    else:
        return 'TEXT'


def fetch_table_list(cursor):
    cursor.execute("SELECT table_name FROM user_tables")
    return [row[0] for row in cursor.fetchall()]


def fetch_columns(cursor, table):
    cursor.execute(f"""
        SELECT column_name, data_type
        FROM user_tab_columns
        WHERE table_name = '{table.upper()}'
        ORDER BY column_id
    """)
    return cursor.fetchall()


def create_pg_table(pg_cursor, table, columns):
    cols_sql = [
        f"{col.lower()} {map_oracle_type(dtype)}"
        for col, dtype in columns
    ]
    ddl = f"CREATE TABLE IF NOT EXISTS {table.lower()} ({', '.join(cols_sql)});"
    print(f"[+] Creating table {table.lower()} in PostgreSQL")
    pg_cursor.execute(ddl)


def migrate_data(ora_cursor, pg_conn, pg_cursor, table, columns):
    col_names = [col.lower() for col, _ in columns]
    col_list = ', '.join(col_names)
    placeholders = ', '.join(['%s'] * len(columns))

    ora_cursor.execute(f"SELECT * FROM {table}")
    rows = ora_cursor.fetchmany(CHUNK_SIZE)

    insert_sql = sql.SQL(
        f"INSERT INTO {table.lower()} ({col_list}) VALUES ({placeholders})"
    )

    total = 0
    while rows:
        pg_cursor.executemany(insert_sql.as_string(pg_conn), rows)
        pg_conn.commit()
        total += len(rows)
        print(f"    ↳ Inserted {total} rows into {table.lower()}")
        rows = ora_cursor.fetchmany(CHUNK_SIZE)


In [16]:

def main():
    # Oracle connection
    ora_conn = cx_Oracle.connect(
        ORACLE["user"], ORACLE["password"], ORACLE["dsn"]
    )
    ora_cursor = ora_conn.cursor()

    # PostgreSQL connection
    pg_conn = psycopg2.connect(**POSTGRES)
    pg_cursor = pg_conn.cursor()

    tables = fetch_table_list(ora_cursor)
    print(f"[✓] Found {len(tables)} Oracle tables.")

    for table in tables:
        columns = fetch_columns(ora_cursor, table)
        create_pg_table(pg_cursor, table, columns)
        migrate_data(ora_cursor, pg_conn, pg_cursor, table, columns)

    print("[✔] Migration complete.")
    ora_cursor.close()
    ora_conn.close()
    pg_cursor.close()
    pg_conn.close()



In [25]:
if __name__ == "__main__":
    main()

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "libclntsh.so: cannot open shared object file: No such file or directory". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

In [22]:
import os

os.environ["ORACLE_HOME"] = "/opt/oracle/instantclient_21_18"
os.environ["LD_LIBRARY_PATH"] = "/opt/oracle/instantclient_21_18"
os.environ["PATH"] += os.pathsep + "/opt/oracle/instantclient_21_18"


In [34]:
import oracledb

# Connect using Thin mode (no Oracle client required)
connection = oracledb.connect(
    user="hr",
    password="hr",
    dsn="localhost:1521/orclpdb1",  # or ORCLCDB
    mode=oracledb.DEFAULT_AUTH
)

print("✅ Connected to Oracle DB")

cursor = connection.cursor()
cursor.execute("SELECT table_name FROM user_tables")
for table_name, in cursor:
    print(" -", table_name)

cursor.close()
connection.close()


✅ Connected to Oracle DB
 - REGIONS
 - LOCATIONS
 - DEPARTMENTS
 - JOBS
 - EMPLOYEES
 - JOB_HISTORY
 - COUNTRIES


In [33]:
!pip install oracledb

57191.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached cffi-1.17.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 1.3 MB/s eta 0:00:00a 0:00:01
Using cached cffi-1.17.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (446 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [oracledb]2/4 [cryptography]
